In [ ]:
import textstat

# On configure textstat pour le français
textstat.set_lang("fr")

def compute_readability_metrics(df, text_col_raw):
    """
    Calcule les scores de lisibilité sur le texte BRUT (ou peu nettoyé).
    """
    # Application du nettoyage léger
    df['text_clean_light'] = df[text_col_raw].apply(clean_for_readability)
    
    # 1. Score de Facilité de Lecture (Flesch Reading Ease)
    # Plus le score est élevé (vers 100), plus c'est facile à lire.
    df['flesch_score'] = df['text_clean_light'].apply(textstat.flesch_reading_ease)
    
    # 2. Niveau scolaire (Flesch-Kincaid Grade)
    # Indique le nombre d'années d'études nécessaires (système US, 8 = 4ème/3ème).
    df['flesch_kincaid_grade'] = df['text_clean_light'].apply(textstat.flesch_kincaid_grade)
    
    # 3. Indice de complexité (Gunning Fog)
    df['gunning_fog'] = df['text_clean_light'].apply(textstat.gunning_fog)

    return df

# --- Synthèse par parti ---
def summarize_readability_by_party(df, party_col='titulaire-soutien'):
    summary = df.groupby(party_col).agg({
        'flesch_score': 'mean',
        'flesch_kincaid_grade': 'mean',
        'gunning_fog': 'mean'
    }).sort_values('flesch_score', ascending=False) # Du plus simple au plus complexe
    
    return summary

summarize_readability_by_party(df_final_sans_cat_mixte)



In [ ]:
import re
import textstat

# On s'assure que textstat utilise les normes françaises
textstat.set_lang("fr")

def clean_for_readability(text):
    if not isinstance(text, str):
        return ""
    # 1. Enlever les sauts de ligne bruyants
    text = re.sub(r'\n+', ' ', text)
    # 2. Garder uniquement lettres, chiffres et ponctuation de base (. , ! ? ;)
    # On retire les symboles bizarres type @, #, *, [ ] qui perturbent textstat
    text = re.sub(r'[^\w\s\.,;!?\-]', '', text)
    # 3. Supprimer les espaces multiples
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Appliquer sur ton texte ORIGINAL (pas sur le texte déjà filtré)
df_final_sans_cat_mixte['text_readability'] = df_final_sans_cat_mixte['text'].apply(clean_for_readability)

In [ ]:
# Score de Flesch (Facilité de lecture)
df_final_sans_cat_mixte['flesch_ease'] = df_final_sans_cat_mixte['text_readability'].apply(textstat.flesch_reading_ease)

# Score de Gunning Fog (Complexité)
df_final_sans_cat_mixte['gunning_fog'] = df_final_sans_cat_mixte['text_readability'].apply(textstat.gunning_fog)

# Nombre de mots (pour vérifier si un texte est trop court pour être fiable)
df_final_sans_cat_mixte['word_count'] = df_final_sans_cat_mixte['text_readability'].apply(textstat.lexicon_count)